## Instructions

Prerequisites:

1. Install the package dependencies found in "requirements.txt".
2. Install an MQTT server, e.g. [Mosquitto](https://mosquitto.org/).


This example is best viewed in jupyter lab. These instructions assume you are running in jupyter lab with the five windows configured as follows:

    +---------------------+---------------+---------------+
    | 1. test-mqtt.ipynb  | 2. terminal   | 3. terminal   |
    |                     | mosquitto     | mosquitto_sub |
    |                     | server        | /spBv1.0/aimpf/DDATA/abar/pycarta
    |                     +---------------+---------------|
    |                     | 4. terminal   | 5. terminal   |
    |                     | mosquitto_sub | mosquitto_sub |
    |                     | test          | /spBv1.0/aimpf/DDATA/bar/pycarta
    +---------------------+---------------+---------------+

That is, the notebook on the left and four terminals: one running the mosquitto server, the other three subscribing to the "test", "/spBv1.0/aimpf/DDATA/abar/pycarta", and "/spBv1.0/aimpf/DDATA/bar/pycarta" topics.

These windows will be referred by number in the comments below.

In [1]:
import time
import logging
import asyncio
from pycarta.mqtt import publish
from pycarta.mqtt import subscribe

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logging.basicConfig()

### Base (pycarta) Publisher

In [2]:
@publish("test", host="localhost")
def foo():
    return "Hello"

In [3]:
# A message should appear in window 4.
foo()

DEBUG:pycarta.mqtt.connection:[Sync Connection] Connecting to localhost:1883
DEBUG:pycarta.mqtt.publisher:Synchronously publishing to topic test
DEBUG:pycarta.mqtt.connection:[Sync Connection] Disconnecting from localhost:1883


'Hello'

In [4]:
@publish("test", host="localhost")
async def afoo():
    await asyncio.sleep(1)  # wait briefly to check if it's truly asynchronous
    return "Async Hello"

In [5]:
await afoo()

'Async Hello'

DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to localhost:1883


DEBUG:pycarta.mqtt.publisher:Asynchronously publishing to topic test
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from localhost:1883


In [ ]:
# A message should appear in window 4.
await asyncio.gather(afoo(), afoo(), afoo())

DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to localhost:1883
DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to localhost:1883
DEBUG:pycarta.mqtt.connection:[Async Connection] Connecting to localhost:1883


['Async Hello', 'Async Hello', 'Async Hello']

DEBUG:pycarta.mqtt.publisher:Asynchronously publishing to topic test
DEBUG:pycarta.mqtt.publisher:Asynchronously publishing to topic test
DEBUG:pycarta.mqtt.publisher:Asynchronously publishing to topic test
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from localhost:1883
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from localhost:1883
DEBUG:pycarta.mqtt.connection:[Async Connection] Disconnecting from localhost:1883


### AIMPF-like Publisher

Notice that the return type/format of the function is unchanged locally, but is formatted according to the CAMX standard when received by the MQTT subscriber.

In [ ]:
@aimpf_publish("my-project", host="localhost")
def bar():
    return "Hello, AIMPF"

@aimpf_publish("my-project", host="localhost")
async def abar():
    # return "Async Hello, AIMPF"
    return {
        "complex": "payload",
        "include": 1.234
    }

In [ ]:
# A JSON-formatted payload should appear in window 5.
bar()

In [ ]:
# A JSON-formatted payload should appear in window 3.
await asyncio.gather(abar(), abar(), abar())

### Base (pycarta) Subscriber

In [1]:
import time
import logging
import asyncio
from pycarta.mqtt import publish
from pycarta.mqtt import subscribe

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logging.basicConfig()

@subscribe("test", host="localhost")
def baz(msg):
    return msg

In [2]:
baz()

DEBUG:pycarta.mqtt.connection:[Sync Connection] Connecting to localhost:1883
INFO:pycarta.mqtt.connection:[Connection] Connecting to localhost:1883...
INFO:pycarta.mqtt.connection:[Connection] Connected to localhost:1883
DEBUG:pycarta.mqtt.subscriber:[Sync subscriber] Subscribing to topic test
DEBUG:pycarta.mqtt.connection:[Sync Connection] Disconnecting from localhost:1883


'Hello from mosquitto 2'

In [ ]:
[m for m in baz]

In [ ]:
import time
import logging
import asyncio
from pycarta.mqtt import publish, aimpf_publish
from pycarta.mqtt import subscribe, aimpf_subscribe

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logging.basicConfig()

@subscribe("test", host="localhost")
async def abaz(msg):
    print(f"[Async Subscriber] Message received: {msg}")
    return msg

In [ ]:
await abaz()

In [ ]:
received_messages = abaz.stop()
print("Received messages:", received_messages)

In [ ]:
received_messages

In [ ]:
await asyncio.gather(abaz(), abaz(), abaz())

In [ ]:
[m async for m in abaz]

In [ ]:
await abaz()

In [ ]:
abaz.stop()

In [ ]:
sub1 = subscribe("test", host="localhost")(abaz)
sub2 = subscribe("test", host="localhost")(abaz)
sub3 = subscribe("test", host="localhost")(abaz)

In [ ]:
sub2.stop()

In [ ]:
@subscribe("test", host="localhost")
async def abaz(msg):
    print(f"[Async Subscriber] Message received: {msg}")
    return msg

sub2 = subscribe("test", host="localhost")(abaz)

In [ ]:
await abaz()

In [ ]:
abaz.stop()

In [ ]:
# Define a subscriber function.
@subscribe("test", host="localhost")
async def my_subscriber(msg):
    print(f"[Subscriber] Received message: {msg}")
    return msg

# Create three independent subscriber instances.
# Even though they wrap the same function, each call returns a separate object.
sub1 = subscribe("test", host="localhost")(my_subscriber)
sub2 = subscribe("test", host="localhost")(my_subscriber)
sub3 = subscribe("test", host="localhost")(my_subscriber)

# Start all subscribers by calling them.
# In our design, calling the subscriber schedules its background listener.
await sub1()
await sub2()
await sub3()
print("All three subscribers are now listening.")

# Let the subscribers run for a while so they can receive messages.
await asyncio.sleep(8)

# Now, stop sub2. This will cancel only sub2's background listener.
sub2.stop()
print("Subscriber 2 stopped.")

# Let the remaining subscribers run a bit longer.
await asyncio.sleep(8)